In [ ]:
%load_ext autoreload
%autoreload 2
from airo_drake.stations.ur3e_cart import MakeUR3eCartStation
from airo_drake.planners.towel_fold_planner import TowelFoldPlanner
from pydrake.geometry import StartMeshcat, MeshcatVisualizer
from pydrake.systems.framework import DiagramBuilder
from pydrake.systems.all import Simulator

In [ ]:
meshcat = StartMeshcat()

In [ ]:
from airo_drake.planners.planners_base import DualArmPlannerBase


station = MakeUR3eCartStation()
plant = station.GetSubsystemByName("plant")
planner = TowelFoldPlanner(plant, meshcat)

In [ ]:
builder = DiagramBuilder()
builder.AddSystem(station)
builder.AddSystem(planner)

# Connect state estimation to planner.
builder.Connect(station.GetOutputPort("ur3e_left_tcp"), planner.GetInputPort("left_tcp"))
builder.Connect(station.GetOutputPort("ur3e_right_tcp"), planner.GetInputPort("right_tcp"))
builder.Connect(station.GetOutputPort("wsg_left_openness_state"), planner.GetInputPort("left_openness_state"))
builder.Connect(station.GetOutputPort("wsg_right_openness_state"), planner.GetInputPort("right_openness_state"))

# Connect planner output to station.
builder.Connect(planner.GetOutputPort("left_tcp_desired"), station.GetInputPort("ur3e_left_tcp_target"))
builder.Connect(planner.GetOutputPort("right_tcp_desired"), station.GetInputPort("ur3e_right_tcp_target"))
builder.Connect(planner.GetOutputPort("left_openness_desired"), station.GetInputPort("wsg_left_openness_target"))
builder.Connect(planner.GetOutputPort("right_openness_desired"), station.GetInputPort("wsg_right_openness_target"))

visualizer = MeshcatVisualizer.AddToBuilder(builder, station.GetOutputPort("query_object"), meshcat)
diagram = builder.Build()

In [ ]:
simulator = Simulator(diagram)
visualizer.StartRecording(False)
simulator.AdvanceTo(6.0)
visualizer.PublishRecording()